In [31]:
import pandas as pd
from huggingface_hub import login
from getpass import getpass
import os
import wandb
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset
from unsloth import FastLanguageModel
import torch

In [32]:
data_path = "/home/wanted-1/potenup-workspace/Project/dacon/DACON-construction-accident-prevention/data/"
train = pd.read_csv(f"{data_path}ref_train.csv", encoding = 'utf-8-sig')
test = pd.read_csv(f"{data_path}test.csv", encoding = 'utf-8-sig')

# 데이터 전처리
train['공사종류(대분류)'] = train['공사종류'].str.split(' / ').str[0]
train['공사종류(중분류)'] = train['공사종류'].str.split(' / ').str[1]
train['공종(대분류)'] = train['공종'].str.split(' > ').str[0]
train['공종(중분류)'] = train['공종'].str.split(' > ').str[1]
train['사고객체(대분류)'] = train['사고객체'].str.split(' > ').str[0]
train['사고객체(중분류)'] = train['사고객체'].str.split(' > ').str[1]

test['공사종류(대분류)'] = test['공사종류'].str.split(' / ').str[0]
test['공사종류(중분류)'] = test['공사종류'].str.split(' / ').str[1]
test['공종(대분류)'] = test['공종'].str.split(' > ').str[0]
test['공종(중분류)'] = test['공종'].str.split(' > ').str[1]
test['사고객체(대분류)'] = test['사고객체'].str.split(' > ').str[0]
test['사고객체(중분류)'] = test['사고객체'].str.split(' > ').str[1]

# 훈련 데이터 통합 생성
combined_training_data = train.apply(
    lambda row: {
        "question": (
            f"공사종류 대분류 '{row['공사종류(대분류)']}', 중분류 '{row['공사종류(중분류)']}' 공사 중 "
            f"공종 대분류 '{row['공종(대분류)']}', 중분류 '{row['공종(중분류)']}' 작업에서 "
            f"사고객체 '{row['사고객체(대분류)']}'(중분류: '{row['사고객체(중분류)']}')와 관련된 사고가 발생했습니다. "
            f"작업 프로세스는 '{row['작업프로세스']}'이며, 사고 원인은 '{row['사고원인']}'입니다. "
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        ),
        "answer": row["재발방지대책 및 향후조치계획"]
    },
    axis=1
)

# DataFrame으로 변환
combined_training_data = pd.DataFrame(list(combined_training_data))

# 테스트 데이터 통합 생성
combined_test_data = test.apply(
    lambda row: {
        "question": (
            f"공사종류 대분류 '{row['공사종류(대분류)']}', 중분류 '{row['공사종류(중분류)']}' 공사 중 "
            f"공종 대분류 '{row['공종(대분류)']}', 중분류 '{row['공종(중분류)']}' 작업에서 "
            f"사고객체 '{row['사고객체(대분류)']}'(중분류: '{row['사고객체(중분류)']}')와 관련된 사고가 발생했습니다. "
            f"작업 프로세스는 '{row['작업프로세스']}'이며, 사고 원인은 '{row['사고원인']}'입니다. "
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        )
    },
    axis=1
)

# DataFrame으로 변환
combined_test_data = pd.DataFrame(list(combined_test_data))

In [33]:
combined_training_data.columns


Index(['question', 'answer'], dtype='object')

In [34]:
# !pip install -q unsloth
# !pip install -q --force-reinstall --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
# 토큰 입력 (비밀번호처럼 입력됨)
hf_token = getpass('Hugging Face 토큰을 입력하세요: ')
login(token=hf_token)

In [49]:
import torch
import gc

# 1️⃣ 현재 사용 중인 모든 CUDA 메모리 해제
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

# 2️⃣ Python이 관리하는 불필요한 객체 수집 (Garbage Collector)
gc.collect()

# 3️⃣ GPU에서 사용 중인 모든 텐서 해제
for obj in gc.get_objects():
    try:
        if torch.is_tensor(obj):
            del obj
    except:
        pass

# 4️⃣ CUDA 컨텍스트 리셋 (가장 강력한 초기화, 그러나 실행 중인 프로세스 죽을 수도 있음)
torch.cuda.reset_max_memory_allocated()
torch.cuda.reset_max_memory_cached()
torch.cuda.synchronize()

In [75]:
# 모델 로드
import torch
from unsloth import FastLanguageModel
from transformers import BitsAndBytesConfig

# ✅ 기본 dtype을 float16으로 설정
torch.set_default_dtype(torch.float16)

# GPU 캐시 정리
torch.cuda.empty_cache()
# 모델 설정값
max_seq_length = 1024
dtype = None  
load_in_4bit = True

# BitsAndBytes 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16, 
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# 모델과 토크나이저 로드
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit",  # 정확한 모델명
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    token=None
)

print("설정이 완료되었습니다!")

==((====))==  Unsloth 2025.3.18: Fast Qwen2 patching. Transformers: 4.49.0.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 2. Max memory: 23.635 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
설정이 완료되었습니다!


In [76]:
# 아래는 작업을 설명하는 지시사항과 추가 맥락을 제공하는 입력이 쌍으로 구성되어 있습니다.
# 요청을 적절히 완료하는 응답을 작성하세요.
# 답변하기 전에 질문을 신중히 생각하고 논리적이고 정확한 응답을 보장하기 위한 단계별 사고 과정을 만드세요.


prompt_style = """
### Instructions: You are a construction safety professional.
Please write a one-sentence answer to the question...
- Answer in Korean
- Do not include any introductions, background, or additional explanations.
- Suggest that the following actions be taken Don't include things like.

{context}

### 질문:
{question}

"""

In [ ]:

question = "공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류 '철근콘크리트공사' 작업에서 사고객체 '건설자재'(중분류: '철근')와 관련된 사고가 발생했습니다. 작업 프로세스는 '설치작업'이며, 사고 원인은 '고소작업 중 추락 위험이 있음에도 불구하고, 안전난간대, 안전고리 착용 등 안전장치가 미흡하였음.'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
# 고소작업 시 추락 위험이 있는 부위에 안전장비 설치.

FastLanguageModel.for_inference(model)
inputs = tokenizer([prompt_style.format(context="", question=question)], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
response_text = response[0].split("### Response:")[1:]  # 리스트 슬라이싱으로 안전한 분할
print("### Response: " + "### Response:".join(response_text))  # 안정적 출력

### Response: 


In [78]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN


def formatting_prompts_func(examples):
    inputs = examples["question"]
    outputs = examples["answer"]
    texts = []
    for input_text, output_text in zip(inputs, outputs):
        text = prompt_style.format(context="", question=input_text) + output_text + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

In [79]:
from datasets import Dataset  # ✅ 올바른 import 방법

# pandas DataFrame을 Dataset으로 변환
dataset = Dataset.from_pandas(combined_training_data)

# 데이터 전처리
dataset = dataset.map(formatting_prompts_func, batched=True)


Map: 100%|██████████| 23422/23422 [00:00<00:00, 134089.21 examples/s]


In [80]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

In [81]:
import os
import wandb
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset

# wandb 비활성화
os.environ["WANDB_DISABLED"] = "true"
wandb.init(mode="disabled")

class CustomDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_length):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        text = item['text']

        # 토크나이징
        encodings = self.tokenizer(
            text,
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt'
        )

        # labels 추가 (input_ids와 동일하게 설정)
        return {
            'input_ids': encodings['input_ids'].squeeze(),
            'attention_mask': encodings['attention_mask'].squeeze(),
            'labels': encodings['input_ids'].squeeze()  # labels 추가
        }

processed_dataset = CustomDataset(dataset, tokenizer, max_seq_length)


training_args = TrainingArguments(
    output_dir="outputs",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=60,
    learning_rate=2e-4,
    fp16=False,   # ✅ float16 활성화
    bf16=True,   # ✅ 가능하면 bfloat16 사용 (더 안정적)
    logging_steps=10,
    optim="adamw_torch",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    report_to="none",
    remove_unused_columns=False,
    gradient_checkpointing=True,  # ✅ VRAM 절약
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset,
)

# ✅ 학습 시작 전 다시 캐시 정리
gc.collect()
torch.cuda.empty_cache()

trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 23,422 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 18,464,768/5,000,000,000 (0.37% trained)


Step,Training Loss
10,10.649500
20,9.664400
30,10.251100
40,10.394400
50,10.278300
60,10.195300


TrainOutput(global_step=60, training_loss=10.238859303792317, metrics={'train_runtime': 111.1403, 'train_samples_per_second': 8.638, 'train_steps_per_second': 0.54, 'total_flos': 9214107089633280.0, 'train_loss': 10.238859303792317, 'epoch': 0.040983606557377046})

In [85]:
question = "공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류 '철근콘크리트공사' 작업에서 사고객체 '건설자재'(중분류: '철근')와 관련된 사고가 발생했습니다. 작업 프로세스는 '설치작업'이며, 사고 원인은 '고소작업 중 추락 위험이 있음에도 불구하고, 안전난간대, 안전고리 착용 등 안전장치가 미흡하였음.'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?"

FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!
inputs = tokenizer([prompt_style.format(context="", question=question)], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
# ✅ 응답이 존재하는지 확인 후 처리
response_text = response[0].split("### Response:")

if len(response_text) > 1:
    response_text = response_text[1].strip()
else:
    response_text = "응답이 없습니다."

print(response_text)

응답이 없습니다.


In [ ]:
new_model_local = "DeepSeek-R1-HS"
model.save_pretrained(new_model_local)
tokenizer.save_pretrained(new_model_local)

model.save_pretrained_merged(new_model_local, tokenizer, save_method = "merged_16bit",)